In [4]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.units import DistanceUnit

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

def get_hf_coords(d):
    return [[0.0, 0.0, 0.0], [d, 0.0, 0.0]]

def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = f"H {coords[0][0]} {coords[0][1]} {coords[0][2]}; F {coords[1][0]} {coords[1][1]} {coords[1][2]}"
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

def run_vqe_gpu(problem, mapper):
    gpu_simulator = AerSimulator(method='statevector', device='GPU', precision='single')
    gpu_simulator.set_options(
        max_parallel_threads=0,
        statevector_parallel_threshold=8,
        blocking_enable=True,
        blocking_qubits=12
    )

    estimator = BackendEstimator(backend=gpu_simulator)
    optimizer = SLSQP(maxiter=300, ftol=1e-6)
    
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result

# --- Main Execution ---
print("=" * 60)
print("HYDROFLUORIC ACID (HF) - GPU-ACCELERATED VQE")
print("=" * 60)

basis = 'sto3g'
charge = 0
spin = 0
dists = np.linspace(0.6, 2.5, 16)

vqe_energies = []

for i, d in enumerate(dists):
    print(f"\n{'='*40}")
    print(f"Calculation {i+1}/{len(dists)}: H-F distance = {d:.2f} Å")
    print(f"{'='*40}")
    
    coords = get_hf_coords(d)
    try:
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)

        vqe_energy, _ = run_vqe_gpu(problem, mapper)
        print(f"VQE energy: {vqe_energy:.8f} Hartree")
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed: {e}")
    
    vqe_energies.append(vqe_energy)

print("\nCompleted all calculations.")


HYDROFLUORIC ACID (HF) - GPU-ACCELERATED VQE

Calculation 1/16: H-F distance = 0.60 Å


/tmp/ipykernel_45414/641638661.py:29: DeprecationWarning: The class ``qiskit.primitives.backend_estimator.BackendEstimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendEstimator` class is `BackendEstimatorV2`.
  estimator = BackendEstimator(backend=gpu_simulator)


KeyboardInterrupt: 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# GPU-enabled imports
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, COBYLA

from pyscf import gto, scf, fci

def get_lih_coords(distance):
    """Returns coordinates for LiH molecule along z-axis"""
    # Li at origin, H at distance along z-axis
    return [[0, 0, 0], [0, 0, distance]]

def get_pyscf_mol(coords, charge, spin, basis):
    """Create PySCF molecule object for LiH"""
    atom_spec = f"Li {coords[0][0]} {coords[0][1]} {coords[0][2]}; H {coords[1][0]} {coords[1][1]} {coords[1][2]}"
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

def get_qiskit_problem(coords, charge, spin, basis):
    """Create Qiskit nature problem for LiH"""
    atom_spec = f"Li {coords[0][0]} {coords[0][1]} {coords[0][2]}; H {coords[1][0]} {coords[1][1]} {coords[1][2]}"
    driver = PySCFDriver(
        atom=atom_spec, 
        unit=DistanceUnit.ANGSTROM, 
        charge=charge, 
        spin=spin, 
        basis=basis
    )
    return driver.run()

def run_rhf_pyscf(mol):
    """Run restricted Hartree-Fock calculation"""
    mf = scf.RHF(mol)
    ehf = mf.kernel()
    return ehf

def run_fci_pyscf(mol):
    """Run Full Configuration Interaction calculation"""
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

def run_vqe_gpu(problem, mapper):
    """GPU-accelerated VQE calculation for LiH"""
    
    # Create GPU-enabled simulator with optimized settings for LiH
    gpu_simulator = AerSimulator(
        method='statevector',
        device='GPU',
        precision='single'
    )
    
    # GPU optimization settings
    gpu_simulator.set_options(
        max_parallel_threads=0,
        statevector_parallel_threshold=8,
        blocking_enable=True,
        blocking_qubits=12,  # Increased for LiH (more qubits than H3+)
        max_memory_mb=8192   # Increased memory allocation
    )
    
    print(f"Using GPU backend: {gpu_simulator.name}")
    
    # Use BackendEstimator with GPU simulator
    estimator = BackendEstimator(backend=gpu_simulator)
    
    # Optimizer settings - COBYLA often works well for VQE
    optimizer = COBYLA(maxiter=300)  # Increased iterations for better convergence
    
    # Prepare ansatz
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(
        problem.num_spatial_orbitals, 
        problem.num_particles, 
        mapper, 
        initial_state=init_state
    )
    
    print(f"Number of qubits: {ansatz.num_qubits}")
    print(f"Number of parameters: {ansatz.num_parameters}")
    print(f"Number of electrons: {problem.num_particles}")
    print(f"Number of spatial orbitals: {problem.num_spatial_orbitals}")
    
    # Initialize VQE
    vqe = VQE(
        estimator, 
        ansatz, 
        optimizer, 
        initial_point=[0.0] * ansatz.num_parameters
    )
    
    # Map Hamiltonian to qubit operators
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    print(f"Starting GPU VQE optimization for LiH...")
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # Extract total energy
    energy = problem.interpret(result).total_energies[0].real
    return energy, result

def check_gpu_availability():
    """Check if GPU is available for Aer simulator"""
    try:
        gpu_simulator = AerSimulator(device='GPU')
        available_devices = gpu_simulator.available_devices()
        print(f"Available devices: {available_devices}")
        if 'GPU' in str(available_devices):
            print("GPU acceleration is available!")
            return True
        else:
            print("GPU not found in available devices")
            return False
    except Exception as e:
        print(f"GPU check failed: {e}")
        return False

# --- Main LiH Calculation ---
print("="*60)
print("LiH Molecule GPU-Accelerated VQE Calculation")
print("="*60)

# Check GPU availability
print("Checking GPU availability...")
gpu_available = check_gpu_availability()

if not gpu_available:
    print("ERROR: GPU not available! This script requires GPU acceleration.")
    print("Please ensure you have:")
    print("1. CUDA-enabled GPU")
    print("2. qiskit-aer-gpu or qiskit-aer with GPU support installed")
    print("3. Proper CUDA drivers installed")
    exit(1)

# LiH calculation parameters
basis = 'sto-3g'  # Start with minimal basis set
charge = 0        # Neutral LiH molecule
spin = 0          # Singlet state (closed shell)

# Bond distances to scan (typical LiH bond length ~1.6 Å)
distances = np.linspace(1.0, 4.0, 16)  # More points around equilibrium

vqe_energies = []
rhf_energies = []
fci_energies = []
vqe_results = []

print(f"\nScanning LiH bond distances from {distances[0]:.1f} to {distances[-1]:.1f} Å")
print(f"Number of distance points: {len(distances)}")

# Loop through different Li-H distances
for i, distance in enumerate(distances):
    print(f"\n{'='*50}")
    print(f"Calculation {i+1}/{len(distances)}: Li-H distance = {distance:.2f} Å")
    print(f"{'='*50}")
    
    coords = get_lih_coords(distance)
    
    # VQE calculation with GPU
    try:
        print("Setting up quantum problem...")
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        print("Running GPU-accelerated VQE...")
        vqe_energy, vqe_result = run_vqe_gpu(problem, mapper)
        vqe_energies.append(vqe_energy)
        vqe_results.append(vqe_result)
        
        print(f"VQE converged: {vqe_result.optimizer_result.success}")
        print(f"Function evaluations: {vqe_result.optimizer_result.nfev}")
        
    except Exception as e:
        vqe_energy = np.nan
        vqe_energies.append(vqe_energy)
        vqe_results.append(None)
        print(f"VQE failed at distance {distance:.2f} Å: {e}")

    # RHF reference calculation (CPU)
    try:
        print("Running RHF reference calculation...")
        mol = get_pyscf_mol(coords, charge, spin, basis)
        rhf_energy = run_rhf_pyscf(mol)
        rhf_energies.append(rhf_energy)
    except Exception as e:
        rhf_energy = np.nan
        rhf_energies.append(rhf_energy)
        print(f"RHF failed at distance {distance:.2f} Å: {e}")

    # FCI reference calculation (CPU)
    try:
        print("Running FCI reference calculation...")
        fci_energy = run_fci_pyscf(mol)
        fci_energies.append(fci_energy)
    except Exception as e:
        fci_energy = np.nan
        fci_energies.append(fci_energy)
        print(f"FCI failed at distance {distance:.2f} Å: {e}")

    print(f"Results: VQE={vqe_energy:.6f} | RHF={rhf_energy:.6f} | FCI={fci_energy:.6f}")

# Convert to numpy arrays for easier manipulation
vqe_energies = np.array(vqe_energies)
rhf_energies = np.array(rhf_energies)
fci_energies = np.array(fci_energies)

# Find minimum energy and equilibrium distance
valid_vqe = ~np.isnan(vqe_energies)
if np.any(valid_vqe):
    min_idx = np.nanargmin(vqe_energies)
    eq_distance = distances[min_idx]
    min_energy = vqe_energies[min_idx]
    print(f"\nVQE Equilibrium distance: {eq_distance:.2f} Å")
    print(f"VQE Minimum energy: {min_energy:.6f} Hartree")

# Plot potential energy curves
plt.figure(figsize=(12, 8))
plt.plot(distances, vqe_energies, 'o-', label='VQE (GPU)', color='blue', linewidth=2, markersize=6)
plt.plot(distances, rhf_energies, 's-', label='RHF (CPU)', color='red', linewidth=2, markersize=5)
plt.plot(distances, fci_energies, '^-', label='FCI (CPU)', color='black', linewidth=2, markersize=5)

plt.xlabel("Li-H Distance (Å)", fontsize=14)
plt.ylabel("Energy (Hartree)", fontsize=14)
plt.title("LiH Potential Energy Curve - GPU-Accelerated VQE", fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Add text box with key results
if np.any(valid_vqe):
    textstr = f'VQE Eq. Distance: {eq_distance:.2f} Å\nVQE Min. Energy: {min_energy:.4f} Ha'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='top', bbox=props)

plt.show()

# Performance and accuracy summary
print(f"\n{'='*60}")
print("CALCULATION SUMMARY")
print(f"{'='*60}")
print(f"Molecule: LiH")
print(f"Basis set: {basis}")
print(f"GPU acceleration: Enabled")
print(f"Successful VQE calculations: {np.sum(~np.isnan(vqe_energies))}/{len(distances)}")

if np.any(valid_vqe) and np.any(~np.isnan(fci_energies)):
    # Calculate errors relative to FCI
    fci_valid = ~np.isnan(fci_energies)
    both_valid = valid_vqe & fci_valid
    if np.any(both_valid):
        vqe_errors = np.abs(vqe_energies[both_valid] - fci_energies[both_valid])
        mean_error = np.mean(vqe_errors)
        max_error = np.max(vqe_errors)
        print(f"VQE vs FCI - Mean absolute error: {mean_error:.6f} Ha")
        print(f"VQE vs FCI - Max absolute error: {max_error:.6f} Ha")

print(f"{'='*60}")

LiH Molecule GPU-Accelerated VQE Calculation
Checking GPU availability...
Available devices: ('CPU', 'GPU')
GPU acceleration is available!

Scanning LiH bond distances from 1.0 to 4.0 Å
Number of distance points: 16

Calculation 1/16: Li-H distance = 1.00 Å
Setting up quantum problem...
Running GPU-accelerated VQE...
Using GPU backend: aer_simulator_statevector_gpu


/tmp/ipykernel_1054/129357542.py:80: DeprecationWarning: The class ``qiskit.primitives.backend_estimator.BackendEstimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendEstimator` class is `BackendEstimatorV2`.
  estimator = BackendEstimator(backend=gpu_simulator)


Number of qubits: 10
Number of parameters: 92
Number of electrons: (2, 2)
Number of spatial orbitals: 6
Starting GPU VQE optimization for LiH...


capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 

In [1]:
import qiskit_aer
from qiskit_aer import AerSimulator

# Check Aer version and GPU support
print(f"Qiskit Aer version: {qiskit_aer.__version__}")

# Test GPU availability
try:
    gpu_sim = AerSimulator(device='GPU')
    devices = gpu_sim.available_devices()
    print(f"Available devices: {devices}")
    
    if 'GPU' in str(devices):
        print("✓ GPU is available for Qiskit")
    else:
        print("✗ GPU not detected")
        
except Exception as e:
    print(f"GPU check failed: {e}")

Qiskit Aer version: 0.17.0
Available devices: ('CPU', 'GPU')
✓ GPU is available for Qiskit


In [2]:
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE, COBYLA
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

# GPU backend
gpu_backend = AerSimulator(method='statevector', device='GPU', precision='single')
gpu_backend.set_options(max_parallel_threads=0, blocking_enable=True)

# Simple VQE problem
hamiltonian = SparsePauliOp.from_list([("ZZ", 1), ("XX", 1)])
ansatz = RealAmplitudes(num_qubits=2, reps=1)

# VQE with GPU
estimator = BackendEstimator(backend=gpu_backend)
vqe = VQE(estimator, ansatz, COBYLA(maxiter=50))

result = vqe.compute_minimum_eigenvalue(hamiltonian)
print(f"Result: {result.eigenvalue:.6f}")

ImportError: cannot import name 'COBYLA' from 'qiskit_algorithms' (/home/enoth/miniconda3/envs/qchem/lib/python3.10/site-packages/qiskit_algorithms/__init__.py)

In [3]:
# Check your current environment
import sys
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_algorithms

print("=== ENVIRONMENT CHECK ===")
print(f"Python version: {sys.version}")
print(f"Qiskit version: {qiskit.__version__}")
print(f"Qiskit Aer version: {qiskit_aer.__version__}")
print(f"Qiskit Nature version: {qiskit_nature.__version__}")
print(f"Qiskit Algorithms version: {qiskit_algorithms.__version__}")

# Check if GPU package is properly installed
try:
    from qiskit_aer import AerSimulator
    print("✓ Qiskit Aer imported successfully")
    
    # Try to create a GPU simulator
    gpu_sim = AerSimulator(device='GPU')
    print("✓ GPU simulator created")
    
    # Check available devices
    devices = gpu_sim.available_devices()
    print(f"Available devices: {devices}")
    
except Exception as e:
    print(f"✗ GPU setup issue: {e}")

=== ENVIRONMENT CHECK ===
Python version: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]
Qiskit version: 1.2.4
Qiskit Aer version: 0.17.0
Qiskit Nature version: 0.7.2
Qiskit Algorithms version: 0.3.0
✓ Qiskit Aer imported successfully
✓ GPU simulator created
Available devices: ('CPU', 'GPU')


In [4]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
import time

print("=== GPU vs CPU COMPARISON ===")

# Create a test circuit
def create_test_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        qc.h(i)
    for i in range(num_qubits-1):
        qc.cx(i, i+1)
    return qc

# Test with different backends
test_circuit = create_test_circuit(6)

# CPU Backend
print("\n--- CPU Backend ---")
cpu_sim = AerSimulator(method='statevector', device='CPU')
start_time = time.time()
cpu_result = cpu_sim.run(test_circuit).result()
cpu_time = time.time() - start_time
print(f"CPU execution time: {cpu_time:.4f} seconds")

# GPU Backend  
print("\n--- GPU Backend ---")
try:
    gpu_sim = AerSimulator(method='statevector', device='GPU')
    start_time = time.time()
    gpu_result = gpu_sim.run(test_circuit).result()
    gpu_time = time.time() - start_time
    print(f"GPU execution time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
except Exception as e:
    print(f"GPU execution failed: {e}")

=== GPU vs CPU COMPARISON ===

--- CPU Backend ---
CPU execution time: 0.0180 seconds

--- GPU Backend ---
GPU execution time: 3.2736 seconds
Speedup: 0.01x


In [5]:
!pip list | grep qiskit

qiskit                   1.2.4
qiskit-aer               0.17.1
qiskit-aer-gpu-cu11      0.17.0
qiskit-algorithms        0.3.0
qiskit-nature            0.7.2


In [6]:
!pip list | grep pyscf

pyscf                    2.9.0
